In [73]:
import os 
import numpy as np
import pandas as pd
import scipy.interpolate as sci

import openpyxl

import matplotlib.pyplot as plt


import jou_gen as jg
import param_postproc as ppproc

from io import StringIO

import plotly.graph_objects as go


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error


import sys
sys.path.append('../RD-systems-and-test-benches/utils')
import plot_functions as pfun

In [74]:
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\1MPE_woch'

df = pd.read_excel(folder_path+'\\1MPE_woch_tui-parametric.xlsx')
df.rename(columns={x : f'VAR_{x}' for x in list(df.columns)[5:]},inplace=True)
df = df.loc[:, ~df.columns.str.startswith('VAR')]

In [75]:
liste = np.arange(51,101)

In [76]:
liste

array([ 51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [77]:
# jg.convert_residuals_csv(folder_path+'\\parametric',liste)
# jg.convert_report(folder_path+'\\parametric','mdot_report',liste)
# jg.convert_report(folder_path+'\\parametric','pressure_report',liste)

In [78]:
residuals_report = {}
mdot_report = {}
pressure_report = {}

for i, row in df.iterrows():
    residuals_report[df.loc[i, 'Name']] = pd.read_csv(folder_path+f"\\parametric\\{df.loc[i, 'Name']}_residuals_report.csv")
    mdot_report[df.loc[i, 'Name']] = pd.read_csv(folder_path+f"\\parametric\\{df.loc[i, 'Name']}_mdot_report.csv")
    pressure_report[df.loc[i, 'Name']] = pd.read_csv(folder_path+f"\\parametric\\{df.loc[i, 'Name']}_pressure_report.csv")    

In [79]:
for i, row in df.iterrows():
    df.loc[i, 'last_continuity_residual'] = residuals_report[df.loc[i, "Name"]].iloc[-1]['continuity']

In [80]:
def set_convergence(row):
    if row['last_continuity_residual'] < 1e-1 and row['last_continuity_residual'] > 1e-2:
        return -1
    elif row['last_continuity_residual'] < 1e-2 and row['last_continuity_residual'] > 1e-3:
        return -2
    elif row['last_continuity_residual'] < 1e-3 and row['last_continuity_residual'] > 1e-4:
        return -3
    elif row['last_continuity_residual'] < 1e-4 and row['last_continuity_residual'] > 1e-5:
        return -4
    # Add more conditions as needed
    elif row['last_continuity_residual'] < 1e-5 and row['last_continuity_residual'] > 1e-6:
        return -5
    elif row['last_continuity_residual'] < 1e-6 and row['last_continuity_residual'] > 1e-7:
        return -6
    # Default value if no condition is met
    return None

df['convergence'] = df.apply(set_convergence, axis=1)

In [81]:
components = mdot_report['DP1']['Component'].unique()
new_columns = [f'DP{i}' for i in range(1, 101)]
mdot_df = pd.DataFrame(index=components, columns=new_columns)

for indew, row in df.iterrows():
    name = row['Name']
    temp = mdot_report[name]
    for component in components:
        mdot_df.at[component, name] = temp.loc[temp['Component'] == component].drop_duplicates()['Mass Flow Rate'].values[0]

In [82]:
for i, row in df.iterrows():
    case = df.loc[i, 'Name']
    df.loc[i, 'sp_coll_inlet'] = pressure_report[case][pressure_report[case]['Component'] == 'coll_inlet']['Value'].values[0]
    df.loc[i, 'sp_coll_outlet'] = pressure_report[case][pressure_report[case]['Component'] == 'coll_outlet']['Value'].values[0]
    df.loc[i, 'sp_distrib_inlet'] = pressure_report[case][pressure_report[case]['Component'] == 'distrib_inlet']['Value'].values[0]
    df.loc[i, 'sp_distrib_outlet'] = pressure_report[case][pressure_report[case]['Component'] == 'distrib_outlet']['Value'].values[0]

In [83]:
df['DPd'] = df['sp_distrib_inlet'] - df['sp_distrib_outlet']
df['DPc'] = df['sp_coll_inlet'] - df['sp_coll_outlet']
df['DPdc'] = df['sp_distrib_outlet'] - df['sp_coll_inlet']

In [84]:
# Define the range of DP values
dp_range = range(1, 101)

# Define the range of coll_ch values
coll_ch_range = range(1, 17)

# Create a Plotly Figure
fig = go.Figure()

# Iterate over DP values
for dp in dp_range:
    # Get the mass flow rate values for the coll_ch values
    coll_ch_mass_flow_rates = [mdot_df.at[f'coll_ch{i}', f'DP{dp}'] for i in coll_ch_range]
    distrib_ch_mass_flow_rates = [-mdot_df.at[f'distrib_ch{i}', f'DP{dp}'] for i in coll_ch_range]

    
    # Add scatter plot to the figure
    fig.add_trace(go.Scatter(x=list(coll_ch_range), y=coll_ch_mass_flow_rates,
                             mode='markers', name=f'DP{dp}_coll_ch'))
    fig.add_trace(go.Scatter(x=list(coll_ch_range), y=distrib_ch_mass_flow_rates,
                                mode='markers', name=f'DP{dp}_distrib_ch'))

# Update layout with labels and title
fig.update_layout(title='Mass Flow Rate for coll_ch1 to coll_ch16',
                  xaxis_title='coll_ch',
                  yaxis_title='Mass Flow Rate',
                  legend_title='DP Values')

# Show the plot
fig.show()

In [85]:
df.rename(columns={'distrib_inlet_mdot/Q_max':'beta'},inplace=True)

df_fil = df[df['Vdot_max'] == 200]
# df_fil = df_fil[df_fil['converged']==True]

In [86]:
df_full = pd.read_excel(r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\1MPE\AbaqueCFD_V4.5_MPE_x1.xlsx')

df_full['DPd'] = df_full['sp_distrib_inlet'] - df_full['sp_distrib_outlet']
df_full['DPc'] = df_full['sp_coll_inlet'] - df_full['sp_coll_outlet']
df_full['DPdc'] = df_full['sp_coll_inlet'] - df_full['sp_distrib_outlet']

df_full_fil = df_full[df_full['Vdot_max'] == 600]

In [87]:
df1d = pd.read_excel(r'G:\Drive partagés\Cercle Hard\Notion\PRJ-1088 - CFD pour la caractérisation hydraulique d’un échangeur à mini-canaux\1D_V4.5_MPE_sans calibrage\Abaque_1MPE_200Lh.xlsx')

In [88]:
X_list = ['beta','alpha']
Y_list = ['DPd','DPc','DPdc']

fig_cfd_list = ppproc.plot_Y_X(df_fil, X_list, Y_list)
fig_1d_list = ppproc.plot_Y_X(df1d, X_list, Y_list)

In [89]:
combined_fig_list = []  # This will store the combined figures

for i in range(len(fig_cfd_list)):
    combined_fig = go.Figure()
    fig = fig_cfd_list[i]
    fig2 = fig_1d_list[i]

    # Extract each trace from this figure and add it to the combined_fig
    for trace in fig.data:
        if trace['mode'] == 'lines':
            mtrace = trace
            mtrace['name'] = mtrace['name'] + " (cfd)"
            combined_fig.add_trace(mtrace)
        else:
            pass
    for trace in fig2.data:
        if trace['mode'] == 'lines':
            mtrace = trace
            mtrace['name'] = mtrace['name'] + " (1d)"
            combined_fig.add_trace(trace)
        else:
            pass

    combined_fig['layout'] = fig['layout']

    combined_fig_list.append(combined_fig)

for fig in combined_fig_list:
    fig.show()

In [90]:
pfun.write_fig_list_one_html(fig_list, len(fig_list)*[""], 'AbaqueCFD_1MPE_600Lh.html')

NameError: name 'fig_list' is not defined

In [ ]:
fig3d = []

X1_name = 'alpha'
X2_name = 'beta'
# Y_name = 'DPd'

for Y_name in ['DPd','DPc','DPdc']:

    dff = df_fil[[X1_name, X2_name, Y_name]]

    # Prepare the data for interpolation
    points = dff[[X1_name, X2_name]].values  # The (X1, X2) pairs
    values = dff[Y_name].values  # The Y values

    # Define the interpolation function
    def interpolate(x1, x2):
        # Single point (x1, x2) for which to interpolate the value
        point = np.array([[x1, x2]])
        # Perform interpolation
        y_interp = sci.griddata(points, values, point, method='linear')
        return y_interp[0]

    # Create a meshgrid for X1_name
    x1_range = np.linspace(0, 1, 100)
    x2_range = np.linspace(0, 1, 100)
    x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)

    # Use the interpolate function over the grid
    y_grid = np.array([interpolate(x1, x2) for x1, x2 in zip(x1_grid.flatten(), x2_grid.flatten())])
    y_grid = y_grid.reshape(x1_grid.shape)

    # X1_name
    fig = go.Figure()

    # Add the surface plot
    fig.add_trace(go.Surface(x=x1_grid, y=x2_grid, z=y_grid, name='Interpolated Surface', colorscale='Viridis'))

    # Add the original data points as scatter plot
    fig.add_trace(go.Scatter3d(x=dff[X1_name], y=dff[X2_name], z=dff[Y_name], mode='markers', 
                            marker=dict(size=5, color='red'), name='Original Data'))

    fig.update_layout(title='3D Surface and Points Plot', autosize=False,
                    width=700, height=700,
                    margin=dict(l=65, r=50, b=65, t=90))

    fig.update_layout(scene = dict(
                        xaxis_title=X1_name,
                        yaxis_title=X2_name,
                        zaxis_title=Y_name))

    fig3d.append(fig)

In [ ]:
pfun.write_fig_list_one_html(fig3d, ['DPd','DPc','DPdc'], '1MPE_600Lh_3D_plots.html')

In [ ]:
df

In [105]:
df.to_excel(r'G:\Drive partagés\Cercle Hard\Notion\PRJ-1088 - CFD pour la caractérisation hydraulique d’un échangeur à mini-canaux\AbaqueCFD_1MPE_woch.xlsx')

In [106]:
# X = np.array([
#     df_fil[X1_name]**2 * df_fil[X2_name]**2,  # X1^2 * X2^2
#     df_fil[X1_name] * df_fil[X2_name]**2,    # X1 * X2^2
#     df_fil[X2_name]**2,               # X2^2
#     df_fil[X1_name]**2 * df_fil[X2_name],    # X1^2 * X2
#     df_fil[X1_name] * df_fil[X2_name],       # X1 * X2
#     df_fil[X2_name],                  # X2
#     df_fil[X1_name]**2,               # X1^2
#     df_fil[X1_name],                  # X1
#     np.ones(len(df_fil))           # Intercept (constant)
# ]).T  # Transpose to get the correct shape

# Y = df_fil[Y_name].values

# # Fit the model
# model = LinearRegression()
# model.fit(X, Y)

# # Coefficients
# a2, b2, c2, a1, b1, c1, a0, b0, c0 = model.coef_

# # Intercept (should be very close to c0 because of the way we structured the equation)
# intercept = model.intercept_

In [107]:
Y_name = 'DPd'

X_columns = df_fil[[X1_name, X2_name]]

# Generate polynomial features up to the third degree
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_columns)

# Fit the model
model = LinearRegression()
model.fit(X_poly, df_fil[Y_name])

# Coefficients
coefficients = model.coef_
intercept = model.intercept_

# Generate a grid for plotting
x1_range = np.linspace(df_fil[X1_name].min(), df_fil[X1_name].max(), 50)
x2_range = np.linspace(df_fil[X2_name].min(), df_fil[X2_name].max(), 50)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)

# Flatten the grid to apply transformations
X_grid = np.vstack([x1_grid.ravel(), x2_grid.ravel()]).T

# Generate polynomial features for the grid
X_grid_poly = poly.transform(X_grid)

# Predict Y values over the grid
y_pred_grid = model.predict(X_grid_poly).reshape(x1_grid.shape)

# Step 1: Generate a grid
x1_range = np.linspace(df_fil[X1_name].min(), df_fil[X1_name].max(), 100)
x2_range = np.linspace(df_fil[X2_name].min(), df_fil[X2_name].max(), 100)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)

# Flatten the grid to apply transformations
X_grid = np.vstack([x1_grid.ravel(), x2_grid.ravel()]).T

# Generate polynomial features for the grid
X_grid_poly = poly.transform(X_grid)

# Predict Y values over the grid
y_pred_grid = model.predict(X_grid_poly).reshape(x1_grid.shape)

# Step 3: Plot the surface
fig = go.Figure()

fig.add_trace(go.Surface(x=x1_grid, y=x2_grid, z=y_pred_grid, name='Interpolated Surface', colorscale='Viridis'))

# Step 4: Overlay the original data points
fig.add_trace(go.Scatter3d(x=df_fil[X1_name], y=df_fil[X2_name], z=df_fil[Y_name], mode='markers', 
                           marker=dict(size=5, color='red'), name='Original Data'))

# Update plot layout
fig.update_layout(title='3D Surface and Points Plot', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene=dict(
                      xaxis_title=X1_name,
                      yaxis_title=X2_name,
                      zaxis_title=Y_name))

# Show the plot
fig.show()

# Step 1: Use the model to predict Y values for the original data points
Y_pred = model.predict(X_poly)  # X_poly contains the polynomial features of the original data

# Step 2: Calculate the differences (errors) between predicted and actual Y values
Y_actual = df_fil[Y_name].values  # Actual Y values from the DataFrame

# Step 3: Compute the RMSE
rmse = np.sqrt(mean_squared_error(Y_actual, Y_pred))

print(f"RMSE: {rmse}")

c:\Users\seagu\.ansys_python_venvs\venv_py_ansys\Lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but PolynomialFeatures was fitted with feature names

c:\Users\seagu\.ansys_python_venvs\venv_py_ansys\Lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but PolynomialFeatures was fitted with feature names



RMSE: 0.17448656603311316
